### API Requests

In [1]:
import os
import json
import base64
import requests

from dotenv import load_dotenv
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [2]:
# Load environment variables
load_dotenv()

api_key = os.getenv("API_KEY")
encryption_key = os.getenv("ENCRYPTION_KEY")

In [3]:
def build_query_string(params: dict) -> str:
    """
    Constructs a URL query string from a dictionary of parameters.

    This function takes a dictionary of key-value pairs and converts them into a
    URL-encoded query string suitable for appending to an API endpoint.

    Args:
        params (dict): A dictionary containing query parameter names and their values.

    Returns:
        str: A URL query string in the format "key1=value1&key2=value2".

    Example:
        >>> params = {"fields": "firstName,lastName", "days_offset": 7}
        >>> build_query_string(params)
        'fields=firstName,lastName&days_offset=7'

    Notes:
        - The keys and values in the dictionary are not automatically encoded. If
          encoding is required (e.g., for spaces or special characters), consider
          using `urllib.parse.urlencode` for more robust handling.
    """
    return "&".join(f"{k}={v}" for k, v in params.items())

### BambooHR API Client

In [4]:
class BambooHRClient:
    """A client for the BambooHR API.
    This client handles authentication, session management, and API requests
    to the BambooHR API or a placeholder API for demonstration purposes.
    """

    DOMAIN = "muttclip"
    BASE_URL = f"https://api.bamboohr.com/api/gateway.php/{DOMAIN}/v1"

    def __init__(self, api_key):
        """Initialize the BambooHRClient with an API key."""
        self.api_key = api_key
        self.base_url = self.BASE_URL
        self.session = self._create_session()
        self.headers = {
            "Content-Type": "application/json",
            "accept": "application/json",
            "Authorization": self._encode_auth_header(api_key),
        }

    def _build_query_string(self, params: dict) -> str:
        """Helper method to construct a query string from a dictionary."""
        return "&".join(f"{k}={v}" for k, v in params.items())

    def _encode_auth_header(self, api_key: str) -> str:
        """Encode the API key in Base64 for the Authorization header."""
        auth_string = f"{api_key}:x"
        encoded_bytes = base64.b64encode(auth_string.encode("utf-8"))
        return f"Basic {encoded_bytes.decode('utf-8')}"

    def _create_session(self) -> requests.Session:
        """Create a session with a retry strategy for handling transient errors."""
        retry_strategy = Retry(
            total=5,
            backoff_factor=2,
            status_forcelist=[429, 503],
            allowed_methods=["GET", "POST"],
        )

        # Create an adapter with the retry strategy
        adapter = HTTPAdapter(max_retries=retry_strategy)

        # Create a session and mount the adapter
        session = requests.Session()
        session.mount("https://", adapter)

        return session

    def get(self, endpoint: str, params: dict = None) -> dict:
        if not endpoint:
            raise ValueError(f"Invalid endpoint: {endpoint}")
        url = f"{self.base_url}{endpoint}"
        response = self.session.get(url, headers=self.headers, params=params)
        try:
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request failed with status {response.status_code}: {e}")
            raise

        return response.json()

    def post(self, endpoint: str, data: dict = None, query_params: dict = None) -> dict:
        """
        Send data to a specified endpoint using the POST method.

        Args:
            endpoint_key (str): The key for the desired endpoint.
            data (dict, optional): The JSON payload to send in the POST request.
            query_params (dict, optional): Query parameters to include in the URL.

        Returns:
            dict: The JSON response from the API.

        Raises:
            ValueError: If an invalid endpoint key is provided.
            requests.exceptions.RequestException: If the request fails.
        """
        if not endpoint:
            raise ValueError(f"Invalid endpoint: {endpoint}")

        if query_params:
            query_string = self._build_query_string(query_params)
            url = f"{self.base_url}{endpoint}?{query_string}"
        else:
            url = f"{self.base_url}{endpoint}"

        response = self.session.post(url, headers=self.headers, json=data)
        try:
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request failed with status {response.status_code}: {e}")
            raise

        return response.json()

In [5]:
from urllib.parse import urlencode


def build_query_string(fields: list) -> str:
    """
    Builds a query string for the employee API request.

    Args:
        fields (list): A list of fields to include in the query string.

    Returns:
        str: The query string to append to the API endpoint.
    """
    query_params = {"fields": ",".join(fields)}
    return f"?{urlencode(query_params)}"

In [ ]:
# Initialize the client
client = BambooHRClient(api_key=api_key)

# Charlotte Abbott
employee_id = 4

field_list = [
    "displayName",
    "firstName",
    "lastName",
    "preferredName",
    "dateOfBirth",
    "maritalStatus",
    "ssn",
    "gender",
    "pronouns",
    "country",
    "city",
    "zipcode",
    "address1",
    "address2",
    "employee_access",
    "customShirtsize",
    "customHobbies",
    "allergies",
    "dietaryRestrictions",
    "jobTitle",
    "hireDate",
    "originalHireDate",
    "employeeStatusDate",
    "employmentStatus",
    "employmentHistoryStatus",  # FTE
    "terminationDate",
    "location",
    "workPhone",
    "mobilePhone",
    "workEmail",
    "department",
    "division",
    "workPhoneExtension",
    "supervisor",
    "supervisorEid",
]

query_params = build_query_string(field_list)
print(query_params)

employee_details = client.get(f"/employees/{employee_id}/{query_params}")

print(json.dumps(employee_details, indent=4))

In [ ]:
client = BambooHRClient(api_key=api_key)

employees_report_payload = {
    "title": "custom-employees",
    "fields": [
        "displayName",
        "firstName",
        "lastName",
        "preferredName",
        "dateOfBirth",
        "maritalStatus",
        "ssn",
        "gender",
        "pronouns",
        "country",
        "city",
        "zipcode",
        "address1",
        "address2",
        "employee_access",
        "customShirtsize",
        "customHobbies",
        "allergies",
        "dietaryRestrictions",
        "jobTitle",
        "hireDate",
        "originalHireDate",
        "employeeStatusDate",
        "employmentStatus",
        "employmentHistoryStatus",  # FTE
        "terminationDate",
        "location",
        "workPhone",
        "mobilePhone",
        "workEmail",
        "department",
        "division",
        "workPhoneExtension",
        "supervisor",
        "supervisorEid",
    ],
}

q_params = {"format": "json", "onlyCurrent": "true"}

custom_employee = client.post(
    "/reports/custom", data=employees_report_payload, query_params=q_params
)

print(json.dumps(custom_employee, indent=4))